# Process

1. Load dataset.
2. Find embeddings.
3. Fit model to different tasks.
4. Vote.
5. Valuation.

# Load datasets

In [2]:
import scripts.exist2024

datasets = scripts.exist2024.load_dataset('EXIST 2024 Memes Dataset')
train = datasets['f_train_dataset.json']
train

In [3]:
# We transform the values into integer encooding
train['score_task5'].astype('category').cat.categories, train['score_task5'].astype('category').cat.codes.unique()

In [4]:
train = datasets['f_train_dataset.json'].encode_memes("EXIST 2024 Memes Dataset", "training", "memes")
train

In [5]:
val = datasets['f_val_dataset.json'].encode_memes("EXIST 2024 Memes Dataset", "training", "memes")
val

# Utility matrix

In [6]:
train.loc[:(6*20)-1].utility_matrix("score_task4")

# Training in both datasets

In [ ]:
# models = train.find_embedding().fit(["score_task4", "score_task5", "score_task6"])
# validation_with_predictions = train.predict(score, models)
import pandas as pd
models = pd.read_parquet('../data/vectorstore.parquet').fit(["score_task4", "score_task5", "score_task6"])
models

In [ ]:
tra

# Find embeddings

In [17]:
import numpy as np
from langchain_experimental.open_clip import OpenCLIPEmbeddings

# Embe images or text
clip_embd = OpenCLIPEmbeddings(model_name="ViT-L-14", checkpoint="laion2b_s32b_b82k")

## Training

In [18]:
embeddings = clip_embd.embed_image(paths)

In [57]:
profile_embeddings = clip_embd.embed_documents(profiles)

In [53]:
import pandas as pd
image_embeddings = pd.concat([paths.reset_index(), pd.Series(embeddings)], axis=1).rename(columns={0: "item_embeddings"})
image_embeddings

In [59]:
import pandas as pd
profile_dataframe_with_embeddings = pd.concat([profiles.reset_index(), pd.Series(profile_embeddings)], axis=1).rename(columns={0: "profile_embeddings"})
profile_dataframe_with_embeddings

In [33]:
image_embeddings.to_parquet('train_dataset_memes_images.parquet')

In [33]:
image_embeddings.to_parquet('train_dataset_memes_images.parquet')

In [68]:
train_with_embeddings = train.merge(image_embeddings[['path', 'item_embeddings']]) \
     .merge(profile_dataframe_with_embeddings[['profile', 'profile_embeddings']])
train_with_embeddings

In [96]:
train_with_embeddings['embeddings'] = train_with_embeddings['profile_embeddings'] + train_with_embeddings['item_embeddings']
train_with_embeddings

In [60]:
profile_dataframe_with_embeddings.to_parquet('train_dataset_memes_profiles.parquet')

In [152]:
X = np.array(train_with_embeddings['embeddings'].tolist())
X.dtype

In [204]:
train_with_embeddings

In [210]:
train_with_embeddings.to_parquet('vectorstore.parquet')

In [211]:
import pandas as pd
pd.read_parquet('vectorstore.parquet')

In [153]:
y = train_with_embeddings['score_task4'].to_numpy()
y

In [157]:
import myfm
from sklearn.feature_extraction import DictVectorizer
import numpy as np

fm = myfm.MyFMRegressor(rank=4)
fm.fit(X,y)


## Validation

In [170]:
val_embeddings = clip_embd.embed_image(val_paths)

In [185]:
val_profile_embeddings = clip_embd.embed_documents(val_profiles)

In [180]:
val_image_embeddings = pd.concat([val_paths.reset_index(), pd.Series(val_embeddings)], axis=1).rename(columns={0: "item_embeddings"})
val_image_embeddings

In [186]:
val_profile_df_embeddings = pd.concat([val_profiles.reset_index(), pd.Series(val_profile_embeddings)], axis=1).rename(columns={0: "profile_embeddings"})
val_profile_df_embeddings

In [202]:
import pandas as pd
val.merge(val_profile_df_embeddings[['profile', 'profile_embeddings']])


In [ ]:
val_with_embeddings.to_parquet('val_dataset_memes.parquet')

In [ ]:
X_val = np.array(val_with_embeddings['embeddings'].tolist())
y_val = np.array(val_with_embeddings['score_task4'].tolist())

# Report Task 4

In [ ]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

predictions = 'EXIST2024_dev_task1_majority_class_hard.json'
gold = tweet_golds("EXIST2024_dev_task1_gold_hard.json")
test = PyEvALLEvaluation()
params= dict()
params[PyEvALLUtils.PARAM_REPORT]= PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED
metrics=["ICM", "ICMNorm" ,"FMeasure", "Accuracy", "Precision", "Recall"]
report= test.evaluate(predictions, gold, metrics, **params)
report.print_report()

# Report Task 5